In [1]:
#Importing libraries
import bempp.api
import numpy as np 
import time
import trimesh
from readoff import *
from readpqr import *
start = time.time()

In [2]:
#Main Data to modify.
#Physical parameters.
em = 4.           #[-] Interior electrical permittivity of the solute.
es = 80.          #[-] Exterior electrical permittivity of the solvent.
ei = (es+em)/2    #[-] Exterior electrical permittivity of the intermediate domain.
ks = 0.125        #[1/A] Inverse of the Debey-Huckel length of the fluid in the solvent.
ki = ks*np.sqrt(es/(es+em))  #[1/A] Inverse of the Debey-Huckel length of the fluid in the intermediate domain.

#Choice of solute.
Mesh1 ='Sphere/Mallas_S/Sphere5R4.off'  #Location of the inner surface mesh of the solute for which the energy is to be calculated. In "off" format
Mesh2 ='Sphere/Mallas_S/Sphere8R4.off'  #Location of the outer surface mesh of the solute for which the energy is to be calculated. In "off" format
PQR = 'Sphere/PQR/Sphere5Q3.pqr'        #Location of the position and charges of the solute for which the energy is to be calculated. In "pqr" format

#Choice to solve the matrix equation in BEM/BEM.
#True: Use the Mass Matrix preconditioner.
#False: No preconditioner is used.
SF = False  #Strong form. 
    
#Assembly of border operators.
#fmm: For molecules with a greater number of vertices.
#default_nonlocal: For molecules with a small number of vertices.    
Assemble = 'default_nonlocal' 

#Important parameters of the GMRES.
Tol =1e-6    #GMRES tolerance.
Res =70      #Restart of the GMRES in each iteration.

#Secondary parameters when working with the fmm assembly.
bempp.api.GLOBAL_PARAMETERS.fmm.expansion_order = 5  
bempp.api.GLOBAL_PARAMETERS.fmm.ncrit = 400          
bempp.api.GLOBAL_PARAMETERS.quadrature.regular = 4  

In [3]:
#Data on the position in space, charge and radii of the atoms of the molecule.
PC,Q,R = readpqr(PQR)   

In [4]:
#Generate the surface meshes of the molecule.
V1,F1 = read_off(Mesh1) 
#In case the mesh has small gaps, with trimesh the information of the original mesh without the gaps is obtained.
meshSP = trimesh.Trimesh(vertices = V1, faces= F1) 
mesh_split = meshSP.split()
print("Found %i meshes"%len(mesh_split)) #1 mesh means no cavity.

vertices_split1 = mesh_split[0].vertices 
faces_split1 = mesh_split[0].faces   
grid1 = bempp.api.grid.grid.Grid(vertices_split1.transpose(), faces_split1.transpose()) #Creation of the inner surface mesh.

verts2,faces2 =read_off(Mesh2) 
grid2 = bempp.api.grid.grid.Grid(verts2.transpose(), faces2.transpose()) #Creation of the outer surface mesh.

Found 1 meshes


In [5]:
#Generate functional spaces of the potential and its derivative for the domain Ωm and Ωi.
dirichl_space1 = bempp.api.function_space(grid1, "P", 1)  #Electrostatic potential at the inner interface.
neumann_space1 = bempp.api.function_space(grid1, "P", 1)  #Derived from the electrostatic potential at the inner interface. 
dirichl_space2 = bempp.api.function_space(grid2, "P", 1)  #Electrostatic potential at the outer interface.
neumann_space2 = bempp.api.function_space(grid2, "P", 1)  #Derived from the electrostatic potential at the outer interface. 
print("DS1 dofs: {0}".format(dirichl_space1.global_dof_count))
print("NS1 dofs: {0}".format(neumann_space1.global_dof_count))
print("DS2 dofs: {0}".format(dirichl_space2.global_dof_count))
print("NS2 dofs: {0}".format(neumann_space2.global_dof_count))

DS1 dofs: 4591
NS1 dofs: 4591
DS2 dofs: 11924
NS2 dofs: 11924


In [6]:
#Generate the boundary operators.
#Identity operators.
I1d = bempp.api.operators.boundary.sparse.identity(dirichl_space1, dirichl_space1, dirichl_space1) # 1
I1n = bempp.api.operators.boundary.sparse.identity(dirichl_space1, neumann_space1, neumann_space1) # 1
I2d = bempp.api.operators.boundary.sparse.identity(dirichl_space2, dirichl_space2, dirichl_space2) # 1
I2n = bempp.api.operators.boundary.sparse.identity(dirichl_space2, neumann_space2, neumann_space2) # 1
#Domain of the solute Ωm.
K111 = bempp.api.operators.boundary.laplace.double_layer(dirichl_space1, dirichl_space1, dirichl_space1, assembler=Assemble) #K
V111 = bempp.api.operators.boundary.laplace.single_layer(neumann_space1, dirichl_space1, dirichl_space1, assembler=Assemble) #V
Y121 = bempp.api.ZeroBoundaryOperator(dirichl_space2, dirichl_space1, dirichl_space1) #0
Z121 = bempp.api.ZeroBoundaryOperator(neumann_space2, dirichl_space1, dirichl_space1) #0
#Intermediate domain Ωi at the inner interface.
if ki==0:
    K211 = bempp.api.operators.boundary.laplace.double_layer(dirichl_space1, neumann_space1, neumann_space1, assembler=Assemble) #K
    V211 = bempp.api.operators.boundary.laplace.single_layer(neumann_space1, neumann_space1, neumann_space1, assembler=Assemble) #V  
    K221 = bempp.api.operators.boundary.laplace.double_layer(dirichl_space2, neumann_space1, neumann_space1, assembler=Assemble) #K
    V221 = bempp.api.operators.boundary.laplace.single_layer(neumann_space2, neumann_space1, neumann_space1, assembler=Assemble) #V
else:
    K211 = bempp.api.operators.boundary.modified_helmholtz.double_layer(dirichl_space1, neumann_space1, neumann_space1, ki, assembler=Assemble) #K
    V211 = bempp.api.operators.boundary.modified_helmholtz.single_layer(neumann_space1, neumann_space1, neumann_space1, ki, assembler=Assemble) #V  
    K221 = bempp.api.operators.boundary.modified_helmholtz.double_layer(dirichl_space2, neumann_space1, neumann_space1, ki, assembler=Assemble) #K
    V221 = bempp.api.operators.boundary.modified_helmholtz.single_layer(neumann_space2, neumann_space1, neumann_space1, ki, assembler=Assemble) #V
#Intermediate domain Ωi at the outer interface.
if ki==0:
    K212 = bempp.api.operators.boundary.laplace.double_layer(dirichl_space1, dirichl_space2, dirichl_space2, assembler=Assemble) #K
    V212 = bempp.api.operators.boundary.laplace.single_layer(neumann_space1, dirichl_space2, dirichl_space2, assembler=Assemble) #V
    K222 = bempp.api.operators.boundary.laplace.double_layer(dirichl_space2, dirichl_space2, dirichl_space2, assembler=Assemble) #K
    V222 = bempp.api.operators.boundary.laplace.single_layer(neumann_space2, dirichl_space2, dirichl_space2, assembler=Assemble) #V
else:
    K212 = bempp.api.operators.boundary.modified_helmholtz.double_layer(dirichl_space1, dirichl_space2, dirichl_space2, ki,  assembler=Assemble) #K
    V212 = bempp.api.operators.boundary.modified_helmholtz.single_layer(neumann_space1, dirichl_space2, dirichl_space2, ki, assembler=Assemble) #V
    K222 = bempp.api.operators.boundary.modified_helmholtz.double_layer(dirichl_space2, dirichl_space2, dirichl_space2, ki, assembler=Assemble) #K
    V222 = bempp.api.operators.boundary.modified_helmholtz.single_layer(neumann_space2, dirichl_space2, dirichl_space2, ki, assembler=Assemble) #V     
#Domain of the solvent Ωs.
Y312 = bempp.api.ZeroBoundaryOperator(dirichl_space1, neumann_space2, neumann_space2) #0
Z312 = bempp.api.ZeroBoundaryOperator(neumann_space1, neumann_space2, neumann_space2) #0
if ks==0:
    K322 = bempp.api.operators.boundary.laplace.double_layer(dirichl_space2, neumann_space2, neumann_space2, assembler=Assemble) #K
    V322 = bempp.api.operators.boundary.laplace.single_layer(neumann_space2, neumann_space2, neumann_space2, assembler=Assemble) #V
else:
    K322 = bempp.api.operators.boundary.modified_helmholtz.double_layer(dirichl_space2, neumann_space2, neumann_space2, ks, assembler=Assemble) #K
    V322 = bempp.api.operators.boundary.modified_helmholtz.single_layer(neumann_space2, neumann_space2, neumann_space2, ks, assembler=Assemble) #V

In [7]:
#Creation of the Coulomb potential function.  
@bempp.api.complex_callable(jit=False)
def U_c(x, n, domain_index, result):
    global Q,PC,em
    result[:] = (1 / (4.*np.pi*em))  * np.sum( Q / np.linalg.norm( x - PC, axis=1))
Uc1 = bempp.api.GridFunction(dirichl_space1, fun=U_c)  

#Construction of the right vector.
if SF==False:
    # Rhs in Ωm.
    rhs_M = (Uc1).projections(dirichl_space1) # uc
    # Rhs in Ωi at inner interface.
    rhs_I1 = np.zeros(neumann_space1.global_dof_count) #0
    # Rhs in Ωi at outer interface.
    rhs_I2 = np.zeros(dirichl_space2.global_dof_count) #0
    # Rhs in Ωs.
    rhs_S = np.zeros(neumann_space2.global_dof_count) #0
    # The combination of Rhs.
    rhs = np.concatenate([rhs_M, rhs_I1, rhs_I2, rhs_S])
else:
    Uc2 = bempp.api.GridFunction(dirichl_space2, fun=U_c) 
    rhs = [I1d*Uc1, 0*I1n*Uc1, 0*I2d*Uc2, 0*I2n*Uc2] 

In [8]:
#Construction left 4x4 matrix.
if SF==False:
    #Position of the 4x4 matrix.
    blocks = [[None,None,None,None],[None,None,None,None],[None,None,None,None],[None,None,None,None]] 
    blocks[0][0] = (0.5*I1d+K111).weak_form()  # 0.5+K  
    blocks[0][1] = -V111.weak_form()           # -V
    blocks[0][2] = Y121.weak_form()            # 0
    blocks[0][3] = Z121.weak_form()            # 0
    blocks[1][0] = (0.5*I1n-K211).weak_form()  # 0.5-K
    blocks[1][1] = (em/ei)*V211.weak_form()    # (em/ei)V
    blocks[1][2] = K221.weak_form()            # K
    blocks[1][3] = -V221.weak_form()           # -V
    blocks[2][0] = -K212.weak_form()           # -K
    blocks[2][1] = (em/ei)*V212.weak_form()    # (em/ei)V
    blocks[2][2] = (0.5*I2d+K222).weak_form()  # 0.5+K
    blocks[2][3] = -V222.weak_form()           # -V
    blocks[3][0] = Y312.weak_form()            # 0
    blocks[3][1] = Z312.weak_form()            # 0
    blocks[3][2] = (0.5*I2n-K322).weak_form()  # 0.5-K
    blocks[3][3] = (ei/es)*V322.weak_form()    # (ei/es)V   
    blocked = bempp.api.assembly.blocked_operator.BlockedDiscreteOperator(np.array(blocks)) 
    #Block diagonal preconditioner for BEM.
    from preconditioners import *
    P = BlockDiagonal_4x4(dirichl_space1, neumann_space1, dirichl_space2, neumann_space2, blocks, es,ei,em,ks,ki)
else:
    blocks = bempp.api.BlockedOperator(4,4)   
    #Position of the 4x4 matrix.
    blocks[0,0] = (0.5*I1d+K111) # 0.5+K 
    blocks[0,1] = -V111          # -V
    blocks[0,2] = Y121           # 0
    blocks[0,3] = Z121           # 0
    blocks[1,0] = (0.5*I1n-K211) # 0.5-K
    blocks[1,1] = (em/ei)*V211   # (em/ei)V
    blocks[1,2] = K221           # K
    blocks[1,3] = -V221          # -V
    blocks[2,0] = -K212          # -K
    blocks[2,1] = (em/ei)*V212   # (em/ei)V
    blocks[2,2] = (0.5*I2d+K222) # 0.5+K
    blocks[2,3] = -V222          # -V
    blocks[3,0] = Y312           # 0
    blocks[3,1] = Z312           # 0
    blocks[3,2] = (0.5*I2n-K322) # 0.5-K
    blocks[3,3] = (ei/es)*V322   # (ei/es)V

In [9]:
#The solution of the matrix equation Ax=B is solved.
#Iteration counter.
it_count = 0
def count_iterations(x):
    global it_count
    it_count += 1
    if (it_count / 100) == (it_count // 100):
        print(it_count,x)    
        
# Solution by GMRES.
from scipy.sparse.linalg import gmres
if SF==False:
    start1 = time.time()
    soln, info = gmres(blocked, rhs, M=P, callback=count_iterations,tol=Tol, restart=Res)  
    end1 = time.time() 
else:
    start1 = time.time()
    soln, info, res, it_count = bempp.api.linalg.gmres(blocks, rhs, return_residuals=True, return_iteration_count=True, use_strong_form=True,tol=Tol, restart=Res)  
    end1 = time.time() 

# Time to solve the equation.
curr_time1 = (end1 - start1)
print("Number of GMRES iterations: {0}".format(it_count))
print("Total time in GMRES: {:5.2f} [s]".format(curr_time1))   

Number of GMRES iterations: 49
Total time in GMRES: 53.45 [s]


In [10]:
#Calculates the entire global domain of the potential from the solution of the calculated both edge.
if SF==False:   
    soln_u1  = soln[:dirichl_space1.global_dof_count]
    soln_du1 = soln[dirichl_space1.global_dof_count : dirichl_space1.global_dof_count + neumann_space1.global_dof_count]
    soln_u2  = soln[dirichl_space1.global_dof_count + neumann_space1.global_dof_count : dirichl_space1.global_dof_count + neumann_space1.global_dof_count + dirichl_space2.global_dof_count]
    soln_du2 = soln[dirichl_space1.global_dof_count + neumann_space1.global_dof_count + dirichl_space2.global_dof_count:]  
    # Solution for Dirichlet data at inner surface.
    dirichlet_fun1 = bempp.api.GridFunction(dirichl_space1, coefficients=soln_u1)
    # Solution for Neumann data at inner surface.
    neumann_fun1 = bempp.api.GridFunction(neumann_space1, coefficients=soln_du1)
    # Solution for Dirichlet data at outer surface.
    dirichlet_fun2 = bempp.api.GridFunction(dirichl_space2, coefficients=soln_u2)
    # Solution for Neumann data at outer surface.
    neumann_fun2 = bempp.api.GridFunction(neumann_space2, coefficients=soln_du2)
else:
    dirichlet_fun1 = soln[0] 
    neumann_fun1 = soln[1] 
    dirichlet_fun2 = soln[2] 
    neumann_fun2 = soln[3]  

In [11]:
#Calculation of the potential in the position of the atoms of the molecule.
VF1 = bempp.api.operators.potential.laplace.single_layer(neumann_space1, np.transpose(PC)) 
KF1 = bempp.api.operators.potential.laplace.double_layer(dirichl_space1, np.transpose(PC))
uF = VF1*neumann_fun1 - KF1*dirichlet_fun1 

#Result of the total solvation energy.
E_Solv = 0.5*4.*np.pi*332.064*np.sum(Q*uF).real 
print('Solvation Energy: {:7.6f} [kCal/mol]'.format(E_Solv) )

#Total time.
end = time.time()
curr_time = (end - start)
print("Total time: {:5.2f} [s]".format(curr_time))

Solvation Energy: -71.152706 [kCal/mol]
Total time: 388.49 [s]
